<a href="https://colab.research.google.com/github/Map987/download-anime-mag-/blob/main/ojbk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Map987/download-anime-mag-.git

Cloning into 'download-anime-mag-'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 76 (delta 5), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (76/76), 2.45 MiB | 3.50 MiB/s, done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

folders = ["/content/sample_data/outputtxt", "/content/sample_data/outtxt", "/content/sample_data/imagebig/"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

In [4]:
import os
import re
import xml.etree.ElementTree as ET
from typing import List

def extract_links(xml_string: str) -> List[str]:
    root = ET.fromstring(xml_string)
    links = []
    for url in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
        image_loc = url.find('{http://www.google.com/schemas/sitemap-image/1.1}image').find('{http://www.google.com/schemas/sitemap-image/1.1}loc').text
        if image_loc:
            links.append(image_loc)
    return links

In [5]:
def write_links_to_file(file_path: str, links: List[str]) -> None:
    with open(file_path, 'w') as f:
        for link in links:
            f.write(link)

In [6]:
input_folder = '/content/download-anime-mag-/'
output_folder = '/content/sample_data/outputtxt/'

for file_name in os.listdir(input_folder):
    if file_name.endswith('.xml'):
        input_path = os.path.join(input_folder, file_name)
        with open(input_path, 'r') as f:
            xml_string = f.read()
        links = extract_links(xml_string)
        output_path = os.path.join(output_folder, file_name[:-4] + '.txt')
        write_links_to_file(output_path, links)
        print(f'{output_path} written successfully.\n')

/content/sample_data/outputtxt/image-sitemap-40.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-37.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-47.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-15.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-41.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-31.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-45.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-51.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-24.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-38.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-11.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-48.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-52.txt written successfully.

/content/sample_data/outp

In [7]:
import os

input_folder_path = '/content/sample_data/outputtxt/'
output_folder_path = '/content/sample_data/outtxt/'

# get all .txt files in input folder
files = [f for f in os.listdir(input_folder_path) if os.path.isfile(os.path.join(input_folder_path, f)) and f.endswith('.txt')]

for file_name in files:
    # open input file
    input_file_path = os.path.join(input_folder_path, file_name)
    with open(input_file_path, 'r') as input_file:
        file_content = input_file.read()

    # add newline before each https
    file_content = file_content.replace('https://', 'https://')

    # create output file
    output_file_path = os.path.join(output_folder_path, file_name)
    with open(output_file_path, 'w') as output_file:
        output_file.write(file_content)

In [8]:
import os

outtxt_dir = "/content/sample_data/outtxt"
data_dir = "/content/sample_data/"

# 遍历outtxt_dir目录中的所有txt文件
for filename in os.listdir(outtxt_dir):
    if filename.endswith(".txt") and filename.startswith("image-sitemap-"):
        # 获取X值
        x = filename.split("-")[-1].split(".")[0]
        
        # 创建imageX文件夹
        folder = os.path.join(data_dir, f"image{x}")
        os.makedirs(folder, exist_ok=True)

In [9]:
import requests
import os
import zipfile
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

def download(url, output_folder_path):
    response = requests.get(url)
    file_name = os.path.basename(url)
    file_path = os.path.join(output_folder_path, file_name)

    with open(file_path, 'wb') as file:
        file.write(response.content)
        print(f"File {file_name} downloaded successfully.")

def process_txt(x):
    url_file_path = f"/content/sample_data/outtxt/image-sitemap-{x}.txt"
    output_folder_path = f"/content/sample_data/image{x}/"
    destination_folder = '/content/sample_data/imagebig'

    try:
        with open(url_file_path, 'r') as url_file:
            file_content = url_file.read()

            urls = file_content.split('https')
            urls = ['https' + url for url in urls if url.endswith('.jpg') or url.endswith('.png')]

            count = len(urls)
            with ThreadPoolExecutor(max_workers=64) as executor:
                futures = [executor.submit(download, url, output_folder_path) for url in urls]
                for future in as_completed(futures):
                    future.result()

        print(f"Downloaded {count} files.")

        source_folder = output_folder_path
        destination_folder = os.path.join(destination_folder, '')
        zip_filename = f"image{x}.zip"
        zip_filepath = os.path.join(destination_folder, zip_filename)

        with zipfile.ZipFile(zip_filepath, 'w') as myzip:
            for filename in os.listdir(source_folder):
                file_path = os.path.join(source_folder, filename)
                myzip.write(file_path, filename)

        print("Files have been successfully zipped to", zip_filepath)

        source_file = zip_filepath
        destination_folder = '/content/drive/MyDrive'

        shutil.copy(source_file, destination_folder)
        print("Zip file has been successfully copied to Google Drive folder.")

    except Exception as e:
        print(f"Error processing {url_file_path}: {e}")

if __name__ == '__main__':
    txt_files = [20]

    for file in txt_files:
        process_txt(file)

File 01-7.jpg downloaded successfully.File 06-5.jpg downloaded successfully.
File 03-7.jpg downloaded successfully.
File 04-5.jpg downloaded successfully.
File 07-1.jpg downloaded successfully.File 05-5.jpg downloaded successfully.

File WAVE_OP4.jpg downloaded successfully.
File WAVE_OP3.jpg downloaded successfully.

File WAVE_OP1.jpg downloaded successfully.
File WAVE_OP2.jpg downloaded successfully.File MNN_03_CAP_0071.jpg downloaded successfully.
File 02-7.jpg downloaded successfully.
File 71736530ac04c50cbaa0939bb48267bc.jpg downloaded successfully.
File d29886408af1ae5c7448e073162a1c44.jpg downloaded successfully.
File Twitter-1-1.jpg downloaded successfully.
File MNN_03_CAP_0067.jpg downloaded successfully.

File d6a3f570835ab307fa8fa16d0e0d4e41-1.jpg downloaded successfully.
File af8615f044cd5308c59ac4901faa8f3a-1.jpg downloaded successfully.
File 7b05b7869f525b7458a3f1b9870357b3.jpg downloaded successfully.File a3c11788bfa0ebbb261c7150e354795d-1.jpg downloaded successfully.
Fi

In [10]:
import requests
import os
import zipfile
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

def download(url, output_folder_path):
    response = requests.get(url)
    file_name = os.path.basename(url)
    file_path = os.path.join(output_folder_path, file_name)

    with open(file_path, 'wb') as file:
        file.write(response.content)
        print(f"File {file_name} downloaded successfully.")

def process_txt(x):
    url_file_path = f"/content/sample_data/outtxt/image-sitemap-{x}.txt"
    output_folder_path = f"/content/sample_data/image{x}/"
    destination_folder = '/content/sample_data/imagebig'

    try:
        with open(url_file_path, 'r') as url_file:
            file_content = url_file.read()

            urls = file_content.split('https')
            urls = ['https' + url for url in urls if url.endswith('.jpg') or url.endswith('.png')]

            count = len(urls)
            with ThreadPoolExecutor(max_workers=64) as executor:
                futures = [executor.submit(download, url, output_folder_path) for url in urls]
                for future in as_completed(futures):
                    future.result()

        print(f"Downloaded {count} files.")

        source_folder = output_folder_path
        destination_folder = os.path.join(destination_folder, '')
        zip_filename = f"image{x}.zip"
        zip_filepath = os.path.join(destination_folder, zip_filename)

        with zipfile.ZipFile(zip_filepath, 'w') as myzip:
            for filename in os.listdir(source_folder):
                file_path = os.path.join(source_folder, filename)
                myzip.write(file_path, filename)

        print("Files have been successfully zipped to", zip_filepath)

        source_file = zip_filepath
        destination_folder = '/content/drive/MyDrive'

        shutil.copy(source_file, destination_folder)
        print("Zip file has been successfully copied to Google Drive folder.")

    except Exception as e:
        print(f"Error processing {url_file_path}: {e}")

if __name__ == '__main__':
    txt_files = [21]

    for file in txt_files:
        process_txt(file)

File 10-7.jpg downloaded successfully.
File 1f57fcf653c9ce10b16fef0528531655.jpg downloaded successfully.
File 20da5223d8d7176a799e2d950fcda57e.jpg downloaded successfully.
File 11-2.jpg downloaded successfully.
File 99c3af265e6a076c78918c8e38791a67.jpg downloaded successfully.
File 0e9210ee5c53181c6355a8cafc033e38.png downloaded successfully.
File HM_logo.png downloaded successfully.
File 9-10.jpg downloaded successfully.
File e3c27d9efd81e2696c67a8527f53c541-1.jpg downloaded successfully.
File 14d81d9c13128d1f8435292066219958-1.jpg downloaded successfully.
File shimono3.jpg downloaded successfully.
File 6-19.jpg downloaded successfully.
File bb98e67b5f15c42b873deaf4fb1d5bee.jpg downloaded successfully.
File 623faf00520267c13354d558dcb05881.jpg downloaded successfully.
File 5bd5fcf3dc7abf740c3c974e2cef4bb1.jpg downloaded successfully.
File 10-8.jpg downloaded successfully.
File bde23ca35e59fda4ed12f6e1668972a8.jpg downloaded successfully.
File eb4a9dc646af24eca08d0e49fa1bf48c-1.jpg do